In [66]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error,r2_score

import pandas_profiling
from pandas_profiling import ProfileReport
from pandas_profiling.utils.cache import cache_file


In [67]:
df=pd.read_csv("C:/Users/Nooshin/Desktop/Data Science/kaggle/CarPrice.csv")

In [68]:
# # print('Pandas profiling version:',pandas_profiling.__version__)
# profile = ProfileReport(df, title="CarPrice Dataset", html={'style': {'full_width': True}}, sort=None)
# profile.to_notebook_iframe()
# profile.to_widgets()

In [69]:
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   car_ID            205 non-null    int64  
 1   symboling         205 non-null    int64  
 2   CarName           205 non-null    object 
 3   fueltype          205 non-null    object 
 4   aspiration        205 non-null    object 
 5   doornumber        205 non-null    object 
 6   carbody           205 non-null    object 
 7   drivewheel        205 non-null    object 
 8   enginelocation    205 non-null    object 
 9   wheelbase         205 non-null    float64
 10  carlength         205 non-null    float64
 11  carwidth          205 non-null    float64
 12  carheight         205 non-null    float64
 13  curbweight        205 non-null    int64  
 14  enginetype        205 non-null    object 
 15  cylindernumber    205 non-null    object 
 16  enginesize        205 non-null    int64  
 1

In [114]:
# df=df.drop(["CarName","wheelbase","car_ID","enginelocation"],axis=1)
df=df.drop(["carlength","carwidth","carheight"],axis=1)
df.head(10)

,symboling,fueltype,aspiration,doornumber,curbweight,cylindernumber,enginesize,boreratio,stroke,compressionratio,...,enginetype_ohcv,enginetype_rotor,fuelsystem_1bbl,fuelsystem_2bbl,fuelsystem_4bbl,fuelsystem_idi,fuelsystem_mfi,fuelsystem_mpfi,fuelsystem_spdi,fuelsystem_spfi
0,1,0,0,0,2548,4,130,3.47,2.68,9.0,...,0,0,0,0,0,0,0,1,0,0
1,1,0,0,0,2548,4,130,3.47,2.68,9.0,...,0,0,0,0,0,0,0,1,0,0
2,3,0,0,0,2823,6,152,2.68,3.47,9.0,...,1,0,0,0,0,0,0,1,0,0
3,2,0,0,1,2337,4,109,3.19,3.40,10.0,...,0,0,0,0,0,0,0,1,0,0
4,2,0,0,1,2824,5,136,3.19,3.40,8.0,...,0,0,0,0,0,0,0,1,0,0
5,2,0,0,0,2507,5,136,3.19,3.40,8.5,...,0,0,0,0,0,0,0,1,0,0
6,3,0,0,1,2844,5,136,3.19,3.40,8.5,...,0,0,0,0,0,0,0,1,0,0
7,3,0,0,1,2954,5,136,3.19,3.40,8.5,...,0,0,0,0,0,0,0,1,0,0
8,3,0,1,1,3086,5,131,3.13,3.40,8.3,...,0,0,0,0,0,0,0,1,0,0
9,4,0,1,0,3053,5,131,3.13,3.40,7.0,...,0,0,0,0,0,0,0,1,0,0


In [115]:
df.symboling.replace({3:1,2:2,1:3,0:4,-1:5,-2:6},inplace=True)
df.head()

,symboling,fueltype,aspiration,doornumber,curbweight,cylindernumber,enginesize,boreratio,stroke,compressionratio,...,enginetype_ohcv,enginetype_rotor,fuelsystem_1bbl,fuelsystem_2bbl,fuelsystem_4bbl,fuelsystem_idi,fuelsystem_mfi,fuelsystem_mpfi,fuelsystem_spdi,fuelsystem_spfi
0,3,0,0,0,2548,4,130,3.47,2.68,9.0,...,0,0,0,0,0,0,0,1,0,0
1,3,0,0,0,2548,4,130,3.47,2.68,9.0,...,0,0,0,0,0,0,0,1,0,0
2,1,0,0,0,2823,6,152,2.68,3.47,9.0,...,1,0,0,0,0,0,0,1,0,0
3,2,0,0,1,2337,4,109,3.19,3.40,10.0,...,0,0,0,0,0,0,0,1,0,0
4,2,0,0,1,2824,5,136,3.19,3.40,8.0,...,0,0,0,0,0,0,0,1,0,0


In [116]:
# L=df[df.enginetype=="rotor"].index
# df.drop(L,axis=0)

In [117]:
def make_one_hot(data,field):
    one_hot=pd.get_dummies(data[field],prefix=field)
    one_hotsum = one_hot.apply(np.sum, axis=1)
    for i in range(data.shape[0]):
        if one_hotsum.iloc[i]==0:
            for j in range(one_hot.shape[1]):
                one_hot.iat[i,j]=np.nan
    data=data.drop(field,axis=1)
    return data.join(one_hot)

In [118]:
df=make_one_hot(df,"carbody")
df=make_one_hot(df,"drivewheel")
df=make_one_hot(df,"enginetype")
df=make_one_hot(df,"fuelsystem")

KeyError: 'carbody'

In [119]:
df.fueltype.replace({"gas":0,"diesel":1},inplace=True)
df.aspiration.replace({"std":0,"turbo":1},inplace=True)
df.doornumber.replace({"two":0,"four":1},inplace=True)
df.cylindernumber.replace({"four":4,"six":6,"five":5,"twelve":12,"three":3,"eight":8,"two":2},inplace=True)

In [120]:
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 38 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   symboling            205 non-null    int64  
 1   fueltype             205 non-null    int64  
 2   aspiration           205 non-null    int64  
 3   doornumber           205 non-null    int64  
 4   curbweight           205 non-null    int64  
 5   cylindernumber       205 non-null    int64  
 6   enginesize           205 non-null    int64  
 7   boreratio            205 non-null    float64
 8   stroke               205 non-null    float64
 9   compressionratio     205 non-null    float64
 10  horsepower           205 non-null    int64  
 11  peakrpm              205 non-null    int64  
 12  citympg              205 non-null    int64  
 13  highwaympg           205 non-null    int64  
 14  price                205 non-null    float64
 15  carbody_convertible  205 non-null    uin

In [124]:
y=df["price"]
x=df.drop("price",axis=1)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1)

In [125]:
SS=StandardScaler()
SS.fit(x_train)
x_train=SS.transform(x_train)
x_test=SS.transform(x_test)

In [126]:
LR = LinearRegression(fit_intercept=True, normalize=False)
LR.fit(x_train,y_train)
y_p=LR.predict(x_test)
print(r2_score(y_test, y_p))
print(mean_absolute_error(y_test, y_p))

0.8506077533830493
2205.7395191147307


C:\Users\Nooshin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(


In [132]:
# import sklearn
# sklearn.metrics.get_scorer_names()

In [134]:
# from sklearn.model_selection import shuffleSplit
from sklearn.model_selection import cross_val_score

# cv=shuffleSplit(n_splits=10,test_size=0.1)
scores=cross_val_score(LR,x,y,cv=5,scoring="r2")
print(scores.mean(), "+/-", scores.std())

0.1235554493240302 +/- 1.2934104980613925


C:\Users\Nooshin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(
C:\Users\Nooshin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(
C:\Users\Nooshin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_bas